In [17]:
import glob
import os
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from shutil import copyfile
import soundfile as sf
from scipy import signal
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import librosa.display
import scipy.io.wavfile

In [18]:
#Loading Data
#Train_Directroy = '.\Train'
try:
    
    os.makedirs('/tmp/wav_files')
    #os.makedirs('/tmp/signals')
    os.makedirs('/tmp/wav_files/signals')
    os.makedirs('/tmp/wav_files/specs')
    os.makedirs('/tmp/noise/noise')
    os.makedirs('/tmp/noise/specs')
    os.makedirs('/tmp/noisy_signals')
    os.makedirs('/tmp/noisy_signals/signals')
    os.makedirs('/tmp/noisy_signals/specs')
    os.makedirs('/tmp/noisy_signals/masks')
    os.makedirs('/tmp/training/input')
    os.makedirs('/tmp/training/output')
    os.makedirs('/tmp/validation/input')
    os.makedirs('/tmp/validation/output')
    os.makedirs('/tmp/testing/input')
    os.makedirs('/tmp/testing/output')
    os.makedirs('/tmp/test_signal/noisy_signal')
    os.makedirs('/tmp/test_signal/spec')
    os.makedirs('/tmp/test_signal/predicted_signal')
    os.makedirs('/tmp/source')
    
except OSError:
    pass

SOURCE_DIR = "/tmp/source/"
TRAINING_DIR = "/tmp/training/"
TESTING_DIR = "/tmp/testing/"
WAV_FILES = "/tmp/wav_files/"


In [19]:
#Load the wav files
SOURCE = "/tmp/source/"   # source of raw data
DEST = "/tmp/wav_files/signals/"
def load_wav_data(SOURCE, DEST):
    list = (os.listdir(SOURCE))
    for i in range(len(list)):
        if(os.path.isdir(SOURCE+list[i])):
            if (os.path.getsize(SOURCE+list[i])>0):
                load_wav_data(SOURCE+list[i]+"/",DEST)     
        elif (list[i].endswith('.WAV')):
            #print(SOURCE+list[i])
            shutil.copy2(SOURCE+list[i],DEST)
            s, fs = sf.read(SOURCE+list[i])
load_wav_data(SOURCE, DEST)


In [20]:
SOURCE = "/tmp/wav_files/signals/"
def get_min_signal(SOURCE):
    minLenSignal = 100000;
    list = (os.listdir(SOURCE))
    for i in range(len(list)):    
        if (list[i].endswith('.WAV')):
            s, fs = sf.read(SOURCE+list[i])
            if(len(s)<minLenSignal):
                minLenSignal = len(s)
    return minLenSignal,fs
minLenSignal,fs = get_min_signal(SOURCE)

In [21]:
print(fs)

16000


In [ ]:
# resize signals to the size of the smallest one
def resize_signals(SOURCE,DEST):
    list = (os.listdir(SOURCE))
    for i in range(len(list)):    
        if (list[i].endswith('.WAV')):
            s, fs = sf.read(SOURCE+list[i])
            for j in range(int(len(s)/minLenSignal)):
                x = s[j*minLenSignal:(j+1)*minLenSignal]
                scipy.io.wavfile.write(DEST+'signal'+str(i+1)+'_'+str(j+1)+'.WAV',fs,x)
#resize_signals(SOURCE,DEST)   

In [21]:
# resample noise signal     
def noise_resample(SOURCE_NOISE,fs):
    b, fb = sf.read(SOURCE_NOISE)
    b = signal.resample(b,int(len(b)*fs/fb))
    return b


In [ ]:
#adding noise to signals
import math
SOURCE_SIG = '/tmp/wav_files/signals/'
SOURCE_NOISE = '/tmp/Bruit/babble.WAV'
DEST_NOISY_SIGNAL = '/tmp/noisy_signals/signals/'
DEST_NOISE = '/tmp/noise/noise/'

def create_noisy_signal(SOURCE,SOURCE_NOISE,DEST_NOISY_SIGNAL,DEST_NOISE,fs):  
    br = noise_resample(SOURCE_NOISE,fs)
    list = (os.listdir(SOURCE))
    list_rsb = []
    for i in range(len(list)):
        print(list[i])
        if (list[i].endswith('.WAV')):
            s, fs = sf.read(SOURCE+list[i])
            num = random.randint(0,(len(br)-len(s))) 
            Ps=sum(s*s)
            Pbr=sum(br[num:(num+len(s))]*br[num:(num+len(s))])
            #alpha= math.sqrt(Ps/(Pbr)) #rsb=0
            #alpha= math.sqrt(Ps/(Pbr*0.25)) #rsb=-6db
            alpha= math.sqrt(Ps/(Pbr*3.98)) #rsb=+6db
            b=alpha*br[num:(num+len(s))]
            Pb=sum(b*b)
            Nsignal = s+b   #Noisy signal
            rsb=10*math.log10(Ps/(Pb))
            list_rsb.append(rsb)
            scipy.io.wavfile.write(DEST_NOISY_SIGNAL+list[i],fs,Nsignal)
            scipy.io.wavfile.write(DEST_NOISE+list[i],fs,b)
    return list_rsb

list_rsb=create_noisy_signal(SOURCE_SIG,SOURCE_NOISE,DEST_NOISY_SIGNAL,DEST_NOISE,fs)
#print(list_rsb)
plt.plot(list_rsb)

In [24]:
# convert signal to spectrogram 
def signal_to_spec(SOURCE, DEST):
    list = (os.listdir(SOURCE))
    for i in range(len(list)):
        if (os.path.getsize(SOURCE+list[i])>0 & list[i].endswith('.WAV')):
            s, fs = sf.read(SOURCE+list[i])
            f_s, t_s, Zss = signal.stft(s, fs, nperseg=512, window='hann', noverlap=256)
            print(Zss.shape)
            #Sdb = librosa.amplitude_to_db(abs(Zss))
            #Sdb_normalized= (Sdb-np.min(Sdb))/(np.max(Sdb)-np.min(Sdb))
            #plt.pcolormesh(t_s, f_s,Sdb_normalized,  shading='auto')
            #librosa.display.specshow(Sdb, sr=fs, x_axis='time', y_axis='hz')
            #plt.title('STFT Magnitude of Sdb')
            #plt.ylabel('Frequency [Hz]')
            #plt.xlabel('Time [sec]')
            with open(DEST+'spec'+list[i][0:-4]+'.npy', 'wb') as f:
                np.save(f, np.array(Zss))
                
            

In [ ]:
# create specs of noise
SOURCE = '/tmp/noise/noise/';
DEST = '/tmp/noise/specs/';            
signal_to_spec(SOURCE, DEST)

# create specs of signals
SOURCE = '/tmp/wav_files/signals/'
DEST = '/tmp/wav_files/specs/'
signal_to_spec(SOURCE, DEST)

# create specs of noisy signals
SOURCE = '/tmp/noisy_signals/signals/'
DEST = '/tmp/noisy_signals/specs/'
signal_to_spec(SOURCE, DEST)

In [26]:
SOURCE_SIGNAL_SPECS = '/tmp/wav_files/specs/'
SOURCE_NOISE_SPECS = '/tmp/noise/specs/'
DEST_MASK = '/tmp/noisy_signals/masks/'

def signal_to_mask(SOURCE_SIGNAL_SPECS,SOURCE_NOISE_SPECS, DEST_MASK):
    list_sig = (os.listdir(SOURCE_SIGNAL_SPECS))
    list_noise = (os.listdir(SOURCE_NOISE_SPECS))
    for i in range(len(list_sig)):
        with open(SOURCE_SIGNAL_SPECS+list_sig[i], 'rb') as f: 
            #SIG = librosa.amplitude_to_db(abs(np.load(f))) 
            SIG = abs(np.load(f))
        with open(SOURCE_NOISE_SPECS+list_noise[i], 'rb') as f:
            #NOISE = librosa.amplitude_to_db(abs(np.load(f)))
            NOISE = abs(np.load(f))
        RSB = (SIG**2)/(NOISE**2)     # parce que amplitude en dB
        where_are_NaNs = np.isnan(RSB)
        RSB[where_are_NaNs] = 1
        RSB[RSB>=0.9]=1
        RSB[RSB<0.9]=0
        Mask = RSB
        """
        print(Mask)
        with open('/tmp/noisy_signals/specs/'+'specSX416.npy', 'rb') as f:
            s = np.load(f)
        print(s.shape)
        t_new,s_new = signal.istft(s*Mask, fs, nperseg=1024, window='hann',nfft= 4096, noverlap=256);
        print(s_new.shape)
        scipy.io.wavfile.write('/tmp'+'/masked_signal.wav',fs,s_new)
        print("eee")
        """
        with open(DEST_MASK+'mask'+list_sig[i][4:-4]+'.npy', 'wb') as f:
            np.save(f, np.array(Mask))
signal_to_mask(SOURCE_SIGNAL_SPECS,SOURCE_NOISE_SPECS, DEST_MASK)

In [ ]:
SOURCE_SPEC = '/tmp/noisy_signals/specs/'
SOURCE_MASK = '/tmp/noisy_signals/masks/'
TRAINING = '/tmp/training/'
TESTING = '/tmp/testing/'
VALIDATION = '/tmp/validation/'
#Split the training set and the test set.
def split_data(SOURCE_SPEC, SOURCE_MASK, TRAINING, TESTING, train_ratio,val_ratio):
    #list_sig = random.sample(os.listdir(SOURCE), len(os.listdir(SOURCE)))
    list_specs = os.listdir(SOURCE_SPEC)
    list_mask = os.listdir(SOURCE_MASK)
    #shuffle data
    ziped_lists = list(zip(list_specs, list_mask))
    random.shuffle(ziped_lists)
    list_specs, list_mask = zip(*ziped_lists)
    
    for i in range(len(list_specs)):
        print(os.path.getsize(SOURCE_MASK+list_mask[i]))
        if(os.path.getsize(SOURCE_SPEC+list_specs[i])>0 and os.path.getsize(SOURCE_MASK+list_mask[i])>0):
            if (i < int(train_ratio*len(list_specs))):
                shutil.copy2(SOURCE_SPEC+list_specs[i],TRAINING+'input/')
                shutil.copy2(SOURCE_MASK+list_mask[i],TRAINING+'output/')
            elif (i < int((train_ratio+val_ratio)*len(list_specs))):
                shutil.copy2(SOURCE_SPEC+list_specs[i],VALIDATION+'input/')
                shutil.copy2(SOURCE_MASK+list_mask[i],VALIDATION+'output/')
            else:
                shutil.copy2(SOURCE_SPEC+list_specs[i],TESTING+'input/')
                shutil.copy2(SOURCE_MASK+list_mask[i],TESTING+'output/')
                
train_ratio = 0.8
val_ratio = 0.1
split_data(SOURCE_SPEC, SOURCE_MASK, TRAINING, TESTING, train_ratio, val_ratio)

In [28]:
def overlapping_indexes(len_array,size,hop):
    #size = 4
    #overlap = 1
    A = list(range(len_array))
    # length = length of the array that we want to split
    # size = size of each resulting chunk
    # hop = number of non overlaping samples between two chunks
    indexes = list(zip(*[A[i:] for i in range(size)]))[::hop]
    return indexes

In [29]:
# Load training and reshape data
def load_reshape_data(SOURCE_DATA):
    
    list_data = os.listdir(SOURCE_DATA+'input/')
    list_labels = os.listdir(SOURCE_DATA+'output/')

    Data = []
    Labels = []
    
    chunk_size = 7
    hop = 1
    
    for i in range(len(list_data)):
        #print(list_data[i])
        with open(SOURCE_DATA+'input/'+list_data[i], 'rb') as f:
            Sdb = librosa.amplitude_to_db(abs(np.load(f)))
            Sdb_normalized= (Sdb-np.min(Sdb))/(np.max(Sdb)-np.min(Sdb))
            indexes = overlapping_indexes(Sdb_normalized.shape[1],chunk_size,hop)
            kmin = int(chunk_size/2)
            kmax = kmin+len(indexes)
            for k in range(kmax-kmin):
                Data.append(Sdb_normalized[:,indexes[k]])

        with open(SOURCE_DATA+'output/'+list_labels[i], 'rb') as f:
            mask = np.load(f)
            split_mask = np.split(mask[:,kmin:kmax],(kmax-kmin), axis=1)
            Labels = Labels+split_mask
            
    data_array = np.array(Data)
    labels_array = np.array(Labels)
    
    data_array = np.transpose(data_array, (0,2,1)) 

    data_array = np.expand_dims(data_array, axis = 3)  
            
    return data_array,labels_array


In [30]:
SOURCE_TRAIN = '/tmp/training/'
input_train_data,output_train_data = load_reshape_data(SOURCE_TRAIN)
print(input_train_data.shape,output_train_data.shape )
with open(SOURCE_TRAIN+'input_train1_data1.npy', 'wb') as f:
    np.save(f, np.array(input_train_data))
with open(SOURCE_TRAIN+'output_train1_data1.npy', 'wb') as f:
    np.save(f, np.array(output_train_data))

(169904, 7, 257, 1) (169904, 257, 1)


In [ ]:
#print(input_train_data[10,:,0:10])
#print(output_train_data[10,0:10,:])

In [31]:
SOURCE_TEST = '/tmp/testing/'
input_test_data,output_test_data = load_reshape_data(SOURCE_TEST) 
print(input_test_data.shape,output_test_data.shape )
with open(SOURCE_TEST+'input_test1_data1.npy', 'wb') as f:
    np.save(f, np.array(input_test_data))
with open(SOURCE_TEST+'output_test1_data1.npy', 'wb') as f:
    np.save(f, np.array(output_test_data))

(22023, 7, 257, 1) (22023, 257, 1)


In [32]:
SOURCE_VAL = '/tmp/validation/'
input_val_data,output_val_data = load_reshape_data(SOURCE_VAL) 
print(input_val_data.shape,output_val_data.shape )
with open(SOURCE_VAL+'input_val1_data1.npy', 'wb') as f:
    np.save(f, np.array(input_val_data))
with open(SOURCE_VAL+'output_val1_data1.npy', 'wb') as f:
    np.save(f, np.array(output_val_data))

(20693, 7, 257, 1) (20693, 257, 1)


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('binary_accuracy')>= 0.95):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True

model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(16, (3,3),padding = 'same', activation='relu',input_shape=(7, 257, 1)),
    tf.keras.layers.Conv2D(32, (3,3), padding = 'same',activation='relu'),
    tf.keras.layers.MaxPooling2D(1,3),
    tf.keras.layers.Conv2D(64, (3,3), padding = 'same',activation='relu'), 
    #tf.keras.layers.Conv2D(32, (3,3), padding = 'same',activation='relu'), 
    tf.keras.layers.MaxPooling2D(1,3),
    #tf.keras.layers.MaxPooling2D(1,12),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1024, activation='relu'), 
    tf.keras.layers.Dense(257, activation='sigmoid'), 
    ])

# Compile Model. 
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
callbacks = myCallback()
# Train the Model
history = model.fit(input_train_data, output_train_data, epochs=30 , verbose = 1,callbacks=[callbacks],validation_data = (input_val_data,output_val_data))

#batch_size =10


In [ ]:
model.save('model')
#model = tf.keras.models.load_model('model')

In [ ]:
print(history.history.keys())

In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['binary_accuracy']
val_acc=history.history['val_binary_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r')#, "Training Accuracy")
plt.plot(epochs, val_acc, 'b')#, "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.legend(['train', 'validation'], loc='upper left')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.figure()


#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r')#, "Training Loss")
plt.plot(epochs, val_loss, 'b')#, "Validation Loss")
plt.title('Training and validation loss')
plt.legend(['train', 'validation'], loc='upper right')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.figure()


In [ ]:
model.evaluate(input_test_data, output_test_data, verbose=1)

In [ ]:
model.summary()

In [ ]:
donnee = input_train_data[0,:,:,:]
donnee = np.expand_dims(donnee, axis = 0)
donnee.shape
Mask = model.predict(donnee)
print(Mask.shape)

In [ ]:
labels = output_train_data[0,:]
sum(sum(abs(Mask-labels)))

In [ ]:
m = tf.keras.metrics.BinaryAccuracy()
m.update_state(labels, Mask)
m.result().numpy()

In [ ]:
# get the spectrogram of the test signal and reshape it
def get_input(SOURCE,chunk_size,hop):
    input_signal = os.listdir(SOURCE)
    Data=[]
    with open(SOURCE+input_signal[0], 'rb') as f:
        SComplex = np.load(f)                                # complex spectrogram
        Sdb = librosa.amplitude_to_db(abs(SComplex))
        Sdb_normalized= (Sdb-np.min(Sdb))/(np.max(Sdb)-np.min(Sdb))
        indexes = overlapping_indexes(Sdb_normalized.shape[1],chunk_size,hop)
        kmin = int(chunk_size/2)
        kmax = kmin+len(indexes)
        for k in range(kmax-kmin):
            #print(B[:,indexes[k]])
            #print(Sdb_normalized[:,indexes[k]].shape)
            Data.append(Sdb_normalized[:,indexes[k]])
        input_data = np.array(Data)
        SComplex = SComplex[:,kmin:kmax]
    input_data = np.transpose(input_data, (0,2,1)) 
    input_data = np.expand_dims(input_data, axis = 3)
    print(input_data.shape)
    print(SComplex.shape)
    return SComplex,input_data

In [ ]:
SOURCE = "/tmp/test_signal/noisy_signal/"
DEST_SPEC = "/tmp/test_signal/spec/"
DEST_SIGNAL = "/tmp/test_signal/predicted_signal"
chunk_size = 7
hop = 1
def denoise_signal(SOURCE,DEST_SPEC,DEST_SIGNAL,chunk_size,hop):
    # create the spectrogram of the test signal
    signal_to_spec(SOURCE, DEST_SPEC)
    spec, input_data = get_input(DEST_SPEC,chunk_size,hop)
    denoised_spec = np.zeros((spec.shape[0],spec.shape[1]),dtype=complex)
    for i in range(input_data.shape[0]):
        spec_chunk = np.expand_dims(input_data[i,:,:,:], axis = 0)
        denoised_chunk = model.predict(spec_chunk)
        #denoised_chunk = denoised_chunk.reshape((chunk_size,-1)).T
        denoised_chunk[denoised_chunk>=0.5] = 1
        denoised_chunk[denoised_chunk<0.5] = 0
        denoised_spec[:,i] = spec[:,i]*denoised_chunk #input_data[i,:,:,0].T
    print(denoised_spec.shape)
    print(input_data[i,:,:,0].T.shape)
    t_new,s_new=signal.istft(denoised_spec, fs, nperseg=512, window='hann',noverlap=256);
    scipy.io.wavfile.write(DEST_SIGNAL+'/signal.wav',fs,s_new)
    return s_new
x = denoise_signal(SOURCE,DEST_SPEC,DEST_SIGNAL,chunk_size,hop)
